## 基于kashgari的命名实体识别

### 安装

前置准备
gensim--3.8.1  
tensorflow--2.5.0           
tensorflow-addons--0.13.0
scikit-learn--0.21.1 
python-- 3.7.1   
pandas--1.2.0 

Kashgari 2.x 安装
通过 github 压缩包下载安装 https://github.com/BrikerMan/Kashgari 

### 准备工作

加载人民日报标注数据

In [21]:
from kashgari.corpus import ChineseDailyNerCorpus

In [22]:
train_x, train_y = ChineseDailyNerCorpus.load_data('train')
valid_x, valid_y = ChineseDailyNerCorpus.load_data('valid')
test_x, test_y = ChineseDailyNerCorpus.load_data('test')

2022-08-31 03:10:05,136 [DEBUG] kashgari - loaded 20864 samples from /home/yuanlin/.kashgari/datasets/china-people-daily-ner-corpus/example.train. Sample:
x[0]: ['据', '说', '，', '小', '马', '尔', '蒂', '尼', '将', '专', '盯', '挪', '威', '队', '的', '尖', '刀', '弗', '洛', '。']
y[0]: ['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O']
2022-08-31 03:10:05,209 [DEBUG] kashgari - loaded 2318 samples from /home/yuanlin/.kashgari/datasets/china-people-daily-ner-corpus/example.dev. Sample:
x[0]: ['工', '作', '了', '一', '段', '时', '间', '后', '他', '又', '表', '示', '想', '接', '受', '培', '训', '，', '于', '是', '她', '又', '帮', '他', '挑', '选', '进', '什', '么', '职', '业', '培', '训', '班', '。']
y[0]: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
2022-08-31 03:10:05,357 [DEBUG] kashgari - loaded 4636 samples from /home/yuanl

In [23]:
print(len(train_x),len(valid_x),len(test_x))

20864 2318 4636


In [24]:
train_x[6],train_y[6]

(['本',
  '报',
  '平',
  '壤',
  '6',
  '月',
  '1',
  '0',
  '日',
  '电',
  '记',
  '者',
  '赵',
  '嘉',
  '鸣',
  '报',
  '道',
  '：',
  '朝',
  '鲜',
  '民',
  '主',
  '主',
  '义',
  '人',
  '民',
  '共',
  '和',
  '国',
  '各',
  '政',
  '党',
  '团',
  '体',
  '代',
  '表',
  '会',
  '议',
  '1',
  '0',
  '日',
  '在',
  '平',
  '壤',
  '举',
  '行',
  '，',
  '会',
  '议',
  '向',
  '韩',
  '国',
  '及',
  '旅',
  '居',
  '海',
  '外',
  '侨',
  '胞',
  '各',
  '政',
  '党',
  '、',
  '团',
  '体',
  '郑',
  '重',
  '建',
  '议',
  '，',
  '于',
  '今',
  '年',
  '8',
  '月',
  '1',
  '4',
  '日',
  '和',
  '1',
  '5',
  '日',
  '朝',
  '鲜',
  '半',
  '岛',
  '解',
  '放',
  '5',
  '3',
  '周',
  '年',
  '之',
  '际',
  '，',
  '在',
  '板',
  '门',
  '店',
  '举',
  '行',
  '由',
  '朝',
  '鲜',
  '半',
  '岛',
  '北',
  '南',
  '双',
  '方',
  '及',
  '海',
  '外',
  '同',
  '胞',
  '共',
  '同',
  '参',
  '加',
  '的',
  '“',
  '争',
  '取',
  '民',
  '族',
  '和',
  '解',
  '、',
  '团',
  '结',
  '和',
  '统',
  '一',
  '大',
  '庆',
  '典',
  '”',
  '。'],
 ['O',
  'O',
  'B-LOC',
  'I-LOC

多种词向量选择：

- BertEmbedding
- WordEmbedding 
- GPT2Embedding

In [25]:
from kashgari.embeddings import BertEmbedding

bert_embed = BertEmbedding('/usr/local/codeData/KG-July/3/chinese_L-12_H-768_A-12')

2022-08-31 03:10:06,198 [DEBUG] kashgari - ------------------------------------------------
2022-08-31 03:10:06,199 [DEBUG] kashgari - Loaded transformer model's vocab
2022-08-31 03:10:06,200 [DEBUG] kashgari - config_path       : /usr/local/codeData/KG-July/3/chinese_L-12_H-768_A-12/bert_config.json
2022-08-31 03:10:06,201 [DEBUG] kashgari - vocab_path      : /usr/local/codeData/KG-July/3/chinese_L-12_H-768_A-12/vocab.txt
2022-08-31 03:10:06,201 [DEBUG] kashgari - checkpoint_path : /usr/local/codeData/KG-July/3/chinese_L-12_H-768_A-12/bert_model.ckpt
2022-08-31 03:10:06,202 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[

可选模型：

- CNN_LSTM_Model
- BiLSTM_Model
- BiGRU_Model
- BiGRU_CRF_Model

In [26]:
from kashgari.tasks.labeling import BiLSTM_CRF_Model

ner_model = BiLSTM_CRF_Model(bert_embed,sequence_length=20)

 ```get_default_hyper_parameters()``` 

```python
hyper = BiLSTM_CRF_Model.default_hyper_parameters()
print(hyper)
#{'layer_blstm': {'units': 128, 'return_sequences': True}, 'layer_dropout': {'rate': 0.4}, 'layer_time_distributed': {}, #'layer_activation': {'activation': 'softmax'}}
hyper["layer_blstm"]["units"] = 100 # 修改参数
ner_model = BiLSTM_CRF_Model(bert_embed,sequence_length=20,hyper_parameters=hyper)
```

模型训练

In [27]:
ner_model.fit(train_x,
          train_y,
          x_validate=valid_x,   
          y_validate=valid_y,
          epochs=5, batch_size=8)

Preparing text vocab dict: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2318/2318 [00:00<00:00, 89378.34it/s]
2022-08-31 03:10:06,481 [DEBUG] kashgari - --- Build vocab dict finished, Total: 3500 ---
2022-08-31 03:10:06,481 [DEBUG] kashgari - Top-10: ['[PAD]', '[UNK]', '[CLS]', '[SEP]', '，', '的', '。', '国', '一', '、']
Preparing text vocab dict: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2318/2318 [00:00<00:00, 142275.51it/s]
2022-08-31 03:10:06,650 [DEBUG] kashgari - --- Build vocab dict finished, Total: 8 ---
2022-08-31 03:10:06,650 [DEBUG] kashgari - Top-10: ['[PAD]', 'O', 'I-ORG', 'I-LOC', 'B-LOC', 'I-PER', 'B-ORG', 'B-PER']
/home/yuanlin/.local/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py:376: UserWarning: CRF decoding models have serialization issues in TF 

Epoch 1/5
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
2608/2608 [==============================] - 735s 275ms/step - loss: 1.9165 - accuracy: 0.9614 - val_loss: 13.6085 - val_accuracy: 0.9722
Epoch 2/5
2608/2608 [==============================] - 715s 274ms/step - loss: 0.8526 - accuracy: 0.9753 - val_loss: 9.8178 - val_accuracy: 0.9317
Epoch 3/5
2608/2608 [==============================] - 712s 273ms/step - loss: 0.6696 - accuracy: 0.9779 - val_loss: 8.4371 - val_accuracy: 0.9272
Epoch 4/5
2608/2608 [==============================] - 707s 271ms/step - loss: 0.5708 - accuracy: 0.9795 - val_loss: 7.8732 - val_accuracy: 0.9193
Epoch 5/5
2608/2608 [==============================] - 705s 270ms/step - loss: 0.5040 - accuracy: 0.9810 - val_loss: 7.7204 - val_accuracy: 0.9185


验证模型，此方法将打印出详细的验证报告：

In [31]:
ner_model.evaluate(test_x, test_y,truncating=True)

2022-08-31 04:11:39,854 [DEBUG] kashgari - predict seq_length: 20, input: (2, 4636, 20)


145/145 [==============================] - 84s 548ms/step


2022-08-31 04:13:04,103 [DEBUG] kashgari - predict output: (4636, 20)
2022-08-31 04:13:04,104 [DEBUG] kashgari - predict output argmax: [[0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 4 3 1]]



           precision    recall  f1-score   support

      LOC     0.8963    0.8910    0.8936      1862
      ORG     0.8855    0.8214    0.8522      1271
      PER     0.9662    0.9601    0.9632      1103

micro avg     0.9118    0.8881    0.8998      4236
macro avg     0.9113    0.8881    0.8993      4236



{'detail': {'LOC': {'precision': 0.8962722852512156,
   'recall': 0.8909774436090225,
   'f1-score': 0.8936170212765958,
   'support': 1862},
  'ORG': {'precision': 0.8854961832061069,
   'recall': 0.8214004720692368,
   'f1-score': 0.8522448979591837,
   'support': 1271},
  'PER': {'precision': 0.9662408759124088,
   'recall': 0.9601087941976428,
   'f1-score': 0.9631650750341064,
   'support': 1103}},
 'precision': 0.911257868301254,
 'recall': 0.8881019830028328,
 'f1-score': 0.8993128509645333,
 'support': 4236}

模型保存：

In [32]:
ner_model.save('ner.h5')

/home/yuanlin/.local/lib/python3.8/site-packages/tensorflow/python/keras/utils/generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
2022-08-31 04:13:06,363 [INFO] kashgari - model saved to /usr/local/CodeProject/CodePro/KG-July/ner.h5


'/usr/local/CodeProject/CodePro/KG-July/ner.h5'

h5 文件是层次数据格式第 5 代的版本（Hierarchical Data Format，HDF5），它是用于存储科学数据的一种文件格式和库文件，由美国超级计算与应用中心研发的文件格式，用以存储和组织大规模数据。目前由非营利组织 HDF 小组提供支持。h5 文件将包含：
- 模型的结构，以便重构该模型
- 模型的权重
- 训练配置（损失函数，优化器等）
- 优化器的状态，以便于从上次训练中断的地方开始

对新输入进行预测：

In [33]:
def ner_predict(input_sen,model):
    ners = model.predict([[char for char in input_sen]])
    return ners

In [34]:
import kashgari
loaded_model = kashgari.utils.load_model('ner.h5')
input_sen = "新华社是个不错的单位"
ners = ner_predict(input_sen,loaded_model)
ners

/tmp/ipykernel_1094519/969091605.py:2: DeprecationWarning: The 'load_model' function is deprecated, use 'XX_Model.load_model' instead
  loaded_model = kashgari.utils.load_model('ner.h5')
2022-08-31 04:13:09,164 [DEBUG] kashgari - ------------------------------------------------
2022-08-31 04:13:09,165 [DEBUG] kashgari - Loaded transformer model's vocab
2022-08-31 04:13:09,166 [DEBUG] kashgari - config_path       : /usr/local/codeData/KG-July/3/chinese_L-12_H-768_A-12/bert_config.json
2022-08-31 04:13:09,167 [DEBUG] kashgari - vocab_path      : /usr/local/codeData/KG-July/3/chinese_L-12_H-768_A-12/vocab.txt
2022-08-31 04:13:09,167 [DEBUG] kashgari - checkpoint_path : /usr/local/codeData/KG-July/3/chinese_L-12_H-768_A-12/bert_model.ckpt
2022-08-31 04:13:09,168 [DEBUG] kashgari - Top 50 words    : ['[PAD]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]

1/1 [==============================] - 5s 5s/step


2022-08-31 04:13:22,775 [DEBUG] kashgari - predict output: (1, 12)
2022-08-31 04:13:22,776 [DEBUG] kashgari - predict output argmax: [[0 6 2 2 1 1 1 1 1 1 1 0]]


[['B-ORG', 'I-ORG', 'I-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O']]